In [1]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .master("local")
    .appName("forecasting")
    .getOrCreate()
)

In [2]:
from pyspark.sql import functions as F

In [3]:
import os

In [4]:
import numpy as np

In [5]:
current_path =  os.getcwd()
raw_data_file_path = './data_raw/train.csv'
raw_data_path = os.path.join(current_path, raw_data_file_path) 
print(f'The path file for the raw data is: {raw_data_path}')

The path file for the raw data is: C:\Users\Admin\Desktop\finally-datascience\./data_raw/train.csv


In [6]:
import warnings
warnings.filterwarnings("ignore")
random_state=1601

In [7]:
historical_df = (
    spark.read.format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load(raw_data_path)
)

In [8]:
# data cruda

In [9]:
historical_df.printSchema()

root
 |-- interval: timestamp (nullable = true)
 |-- total_calls: integer (nullable = true)
 |-- total_calls_duration: integer (nullable = true)
 |-- missing_calls: integer (nullable = true)
 |-- available_time: integer (nullable = true)
 |-- away_time: integer (nullable = true)
 |-- busy_time: integer (nullable = true)
 |-- on_a_call_time: integer (nullable = true)
 |-- after_call_work_time: integer (nullable = true)
 |-- total_handle_time: integer (nullable = true)
 |-- occupancy_rate: double (nullable = true)
 |-- utilization_rate: double (nullable = true)
 |-- shrinkage_rate: double (nullable = true)
 |-- agent_headcount: integer (nullable = true)



In [10]:
historical_df.createOrReplaceTempView('historico_df')

In [11]:
historical_df.toPandas()

,interval,total_calls,total_calls_duration,missing_calls,available_time,away_time,busy_time,on_a_call_time,after_call_work_time,total_handle_time,occupancy_rate,utilization_rate,shrinkage_rate,agent_headcount
0,2017-01-01 00:00:00,227,11880,8,9721,28806,0,14047,4543,18590,49.566679,32.547228,50.433321,56
1,2017-01-01 00:15:00,235,15297,4,10716,34697,0,16636,4932,21568,48.198743,32.200176,51.801257,60
2,2017-01-01 00:30:00,219,13739,5,16380,24186,0,16092,6280,22372,61.571705,35.546093,38.428295,61
3,2017-01-01 00:45:00,256,16534,5,8086,12307,0,16253,5932,22185,71.095401,52.104373,28.904599,57
4,2017-01-01 01:00:00,240,17692,14,8371,25296,0,18225,7217,25442,57.204487,43.042515,42.795513,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67099,2018-11-30 22:45:00,673,93391,2,10799,96238,217301,72372,19346,91718,24.640193,22.044629,75.359807,273
67100,2018-11-30 23:00:00,676,100549,9,17564,100986,182983,82095,20250,102345,29.689411,25.340573,70.310589,283
67101,2018-11-30 23:15:00,717,122798,18,27736,104007,244592,70592,42260,112852,28.739112,23.069297,71.260888,298
67102,2018-11-30 23:30:00,747,116056,13,11577,112657,267705,89426,21767,111193,24.401151,22.100165,75.598849,293


In [12]:
query = """
select to_date(DATEADD(month, -1, to_date(interval))) as fecha, sum(total_calls) as total_calls 
from historico_df
group by to_date(interval)
"""

In [13]:
spine_df = spark.sql(query)

In [14]:
spine_df.show()

+----------+-----------+
|     fecha|total_calls|
+----------+-----------+
|2017-07-11|      15462|
|2017-08-11|      13963|
|2018-04-28|      27654|
|2018-07-10|      29415|
|2016-12-06|       7763|
|2018-02-17|      23773|
|2018-05-06|      26749|
|2018-05-26|      27475|
|2018-07-08|      26150|
|2018-07-11|      28879|
|2018-08-01|      29744|
|2018-08-09|      33183|
|2018-09-05|      31342|
|2018-10-02|      32771|
|2016-12-27|       8384|
|2017-01-26|       7338|
|2017-08-28|      14476|
|2018-05-30|      29951|
|2016-12-24|       6673|
|2018-10-01|      28996|
+----------+-----------+
only showing top 20 rows



In [15]:
historical_df = historical_df.withColumn('fecha', F.to_date(F.col('interval')))

In [16]:
historical_df.toPandas()

,interval,total_calls,total_calls_duration,missing_calls,available_time,away_time,busy_time,on_a_call_time,after_call_work_time,total_handle_time,occupancy_rate,utilization_rate,shrinkage_rate,agent_headcount,fecha
0,2017-01-01 00:00:00,227,11880,8,9721,28806,0,14047,4543,18590,49.566679,32.547228,50.433321,56,2017-01-01
1,2017-01-01 00:15:00,235,15297,4,10716,34697,0,16636,4932,21568,48.198743,32.200176,51.801257,60,2017-01-01
2,2017-01-01 00:30:00,219,13739,5,16380,24186,0,16092,6280,22372,61.571705,35.546093,38.428295,61,2017-01-01
3,2017-01-01 00:45:00,256,16534,5,8086,12307,0,16253,5932,22185,71.095401,52.104373,28.904599,57,2017-01-01
4,2017-01-01 01:00:00,240,17692,14,8371,25296,0,18225,7217,25442,57.204487,43.042515,42.795513,61,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67099,2018-11-30 22:45:00,673,93391,2,10799,96238,217301,72372,19346,91718,24.640193,22.044629,75.359807,273,2018-11-30
67100,2018-11-30 23:00:00,676,100549,9,17564,100986,182983,82095,20250,102345,29.689411,25.340573,70.310589,283,2018-11-30
67101,2018-11-30 23:15:00,717,122798,18,27736,104007,244592,70592,42260,112852,28.739112,23.069297,71.260888,298,2018-11-30
67102,2018-11-30 23:30:00,747,116056,13,11577,112657,267705,89426,21767,111193,24.401151,22.100165,75.598849,293,2018-11-30


In [17]:
(
    historical_df.withColumn('fecha', F.to_date(F.col('interval')))
              .select('interval', 'fecha')
              .limit(2)
              .show()
)

+-------------------+----------+
|           interval|     fecha|
+-------------------+----------+
|2017-01-01 00:00:00|2017-01-01|
|2017-01-01 00:15:00|2017-01-01|
+-------------------+----------+



In [18]:
historical_df.select('interval', 'fecha').limit(2).show()

+-------------------+----------+
|           interval|     fecha|
+-------------------+----------+
|2017-01-01 00:00:00|2017-01-01|
|2017-01-01 00:15:00|2017-01-01|
+-------------------+----------+



In [19]:
historical_df.select(F.col('fecha')).distinct()

DataFrame[fecha: date]

In [20]:
spine_df =  historical_df.select(F.col('fecha')).distinct()

target_df = (
    historical_df.groupBy(F.col('fecha'))
    .agg(F.sum(F.col('total_calls')).cast('int').alias('total_calls'))
    .withColumn('fecha', F.add_months(F.col('fecha'), -1))
)

In [21]:
target_df.sort(F.col('fecha')).toPandas()

,fecha,total_calls
0,2016-12-01,8672
1,2016-12-02,8324
2,2016-12-03,6527
3,2016-12-04,6939
4,2016-12-05,6904
...,...,...
694,2018-10-26,29245
695,2018-10-27,29577
696,2018-10-28,29885
697,2018-10-29,31252


In [22]:
spine_df.sort(F.col('fecha')).toPandas()

,fecha
0,2017-01-01
1,2017-01-02
2,2017-01-03
3,2017-01-04
4,2017-01-05
...,...
694,2018-11-26
695,2018-11-27
696,2018-11-28
697,2018-11-29


In [23]:
historical_df.columns

['interval',
 'total_calls',
 'total_calls_duration',
 'missing_calls',
 'available_time',
 'away_time',
 'busy_time',
 'on_a_call_time',
 'after_call_work_time',
 'total_handle_time',
 'occupancy_rate',
 'utilization_rate',
 'shrinkage_rate',
 'agent_headcount',
 'fecha']

In [24]:
def create_dataframe_from_schema(schema):
    empty_RDD = spark.sparkContext.emptyRDD()
    return spark.createDataFrame(empty_RDD, schema)

In [25]:
keys = [F.col("df_publico.fecha") >= F.col("df_historico.fecha")]

full_table_df = (
    spine_df.alias('df_publico')
    .join(historical_df.alias('df_historico'), how='left', on=keys)
    .withColumn('diff_days', F.datediff('df_publico.fecha', 'df_historico.fecha'))
)

In [26]:
from pyspark.sql.types import *
from pyspark.sql import DataFrame


dias_comparar = [3, 7, 15]
columns = [
 'total_calls_historical',
 'total_calls_duration',
 'missing_calls',
 'available_time',
 'away_time',
 'busy_time',
 'on_a_call_time',
 'after_call_work_time',
 'total_handle_time',
 'occupancy_rate',
 'utilization_rate',
 'shrinkage_rate',
 'agent_headcount'
]
fecha_grouped = create_dataframe_from_schema(
    StructType(
            [
                StructField("fecha", DateType(), True)
            ]
    )
)

keys = [F.col("df_publico.fecha") >= F.col("df_historico.fecha")]

full_table_df = (
    spine_df.alias('df_publico')
    .join(historical_df.withColumnRenamed('total_calls', 'total_calls_historical').alias('df_historico'), how='left', on=keys)
    .withColumn('diff_days', F.datediff('df_publico.fecha', 'df_historico.fecha'))
)
    
for days_i in dias_comparar:
    for column_i in columns:
        fecha_grouped_ = (
            full_table_df.filter(F.col("diff_days") <= days_i)
                .groupBy(
                    "df_publico.fecha"
                ).agg(
                    (
                        F.mean(
                            F.col(column_i)
                        )
                    ).alias(f"mean_{column_i}_last_" + str(days_i) + "_days")
                    ,
                    (
                        F.max(
                            F.col(column_i)
                        )
                    ).alias(f"max_{column_i}_last_" + str(days_i) + "_days")
                    ,
                    (
                        F.stddev(
                            F.col(column_i)
                        )
                        
                    ).alias(f'stddev_{column_i}_last_{str(days_i)}_days'),
                    (
                         F.variance(
                             F.col(column_i)
                         ).alias(f"var_{column_i}_last_" + str(days_i) + "_days")
                     ),
                    (
                        F.min(
                            F.col(column_i)
                        )
                    ).alias(f"min_{column_i}_last_" + str(days_i) + "_days")
                )
        )
        fecha_grouped = fecha_grouped.join(
            fecha_grouped_, on=["fecha"], how="full"
        )


In [27]:
fecha_grouped.limit(10).toPandas()

,fecha,mean_total_calls_historical_last_3_days,max_total_calls_historical_last_3_days,stddev_total_calls_historical_last_3_days,var_total_calls_historical_last_3_days,min_total_calls_historical_last_3_days,mean_total_calls_duration_last_3_days,max_total_calls_duration_last_3_days,stddev_total_calls_duration_last_3_days,var_total_calls_duration_last_3_days,...,mean_shrinkage_rate_last_15_days,max_shrinkage_rate_last_15_days,stddev_shrinkage_rate_last_15_days,var_shrinkage_rate_last_15_days,min_shrinkage_rate_last_15_days,mean_agent_headcount_last_15_days,max_agent_headcount_last_15_days,stddev_agent_headcount_last_15_days,var_agent_headcount_last_15_days,min_agent_headcount_last_15_days
0,2017-01-01,90.333333,266,88.514247,7834.771930,0,5967.968750,18631,5940.229777,3.528633e+07,...,30.150837,98.766546,26.121015,682.307405,0.0,26.260417,71,24.474797,599.015680,0
1,2017-01-02,88.520833,266,84.239434,7096.282286,0,6257.140625,22622,6156.384229,3.790107e+07,...,31.303106,98.766546,25.530958,651.829841,0.0,28.093750,72,25.268227,638.483312,0
2,2017-01-03,81.677083,266,78.232945,6120.393619,0,6384.482639,22622,6245.698861,3.900875e+07,...,31.010470,98.766546,25.222589,636.178973,0.0,27.822917,72,24.901756,620.097452,0
3,2017-01-04,79.328125,266,75.241555,5661.291531,0,6516.236979,28632,6367.959209,4.055090e+07,...,30.961792,98.766546,25.081738,629.093594,0.0,27.843750,72,25.042034,627.103460,0
4,2017-01-05,74.723958,220,69.164592,4783.740834,0,6749.882812,28632,6538.989143,4.275838e+07,...,30.869730,98.766546,25.010592,625.529709,0.0,28.154167,73,25.314168,640.807081,0
5,2017-01-06,73.263021,224,67.603483,4570.230901,0,6928.864583,28632,6691.416800,4.477506e+07,...,30.492465,98.766546,24.695209,609.853365,0.0,28.585069,81,25.840441,667.728403,0
6,2017-01-07,75.848958,224,69.960922,4894.530652,0,6779.184896,28632,6526.920275,4.260069e+07,...,30.807910,98.766546,24.692624,609.725672,0.0,28.049107,81,25.351402,642.693561,0
7,2017-01-08,75.153646,224,70.601813,4984.616018,0,6389.294271,23753,6217.190040,3.865345e+07,...,30.577533,98.766546,24.652693,607.755262,0.0,27.291667,81,24.774252,613.763581,0
8,2017-01-09,75.588542,224,70.194266,4927.234960,0,6257.880208,23753,6033.113690,3.639846e+07,...,30.267264,98.766546,24.539512,602.187658,0.0,27.282407,81,24.655238,607.880756,0
9,2017-01-10,73.812500,220,67.967442,4619.573107,0,6189.869792,23576,5965.000931,3.558124e+07,...,30.185635,98.766546,24.497423,600.123734,0.0,27.308333,81,24.682610,609.231213,0


In [28]:
new_spine_df = (
    spine_df.join(target_df, how='inner', on='fecha')
    .join(fecha_grouped,how='left', on='fecha')
)

In [29]:
fecha_inicial = '2017-01-16' 
fecha_final = '2018-09-30'  

In [30]:
train = new_spine_df.filter(F.col('fecha').between(fecha_inicial, fecha_final))
test = new_spine_df.filter(F.col('fecha') > fecha_final)

In [31]:
train_fe_df = train.sort(F.col('fecha')).toPandas()
test_fe_df = test.sort(F.col('fecha')).toPandas()

In [32]:
# Specify the output directory
output_directory = os.path.expanduser('./generated_data')  
train_fe_df.to_csv(os.path.join(output_directory, 'train.csv'), header=True, index=False)
test_fe_df.to_csv(os.path.join(output_directory, 'test.csv'), header=True, index=False)
"""
#Diego: (kaggle)
train_fe_df.write.mode('overwrite').csv('train.csv',header = 'true')
test_df.write.mode('overwrite').csv('test.csv',header = 'true')
"""

"\n#Diego: (kaggle)\ntrain_fe_df.write.mode('overwrite').csv('train.csv',header = 'true')\ntest_df.write.mode('overwrite').csv('test.csv',header = 'true')\n"

In [33]:
train_fe_df

,fecha,total_calls,mean_total_calls_historical_last_3_days,max_total_calls_historical_last_3_days,stddev_total_calls_historical_last_3_days,var_total_calls_historical_last_3_days,min_total_calls_historical_last_3_days,mean_total_calls_duration_last_3_days,max_total_calls_duration_last_3_days,stddev_total_calls_duration_last_3_days,...,mean_shrinkage_rate_last_15_days,max_shrinkage_rate_last_15_days,stddev_shrinkage_rate_last_15_days,var_shrinkage_rate_last_15_days,min_shrinkage_rate_last_15_days,mean_agent_headcount_last_15_days,max_agent_headcount_last_15_days,stddev_agent_headcount_last_15_days,var_agent_headcount_last_15_days,min_agent_headcount_last_15_days
0,2017-01-16,7965,75.854167,246,71.689274,5139.352045,0,6256.955729,22407,6060.377079,...,29.815941,99.907664,24.354658,593.149376,0.0,27.192708,82,24.550433,602.723751,0
1,2017-01-17,8075,73.312500,246,68.251044,4658.204961,0,6168.911458,22407,5970.096135,...,29.777876,99.907664,24.258832,588.490925,0.0,27.274089,82,24.538922,602.158703,0
2,2017-01-18,8166,71.859375,216,65.952242,4349.698189,0,6544.958333,25847,6358.409732,...,29.568132,99.950187,24.171977,584.284472,0.0,27.168620,82,24.473869,598.970246,0
3,2017-01-19,7379,73.078125,210,66.294021,4394.897275,0,7161.338542,25847,6832.182531,...,29.292342,99.950187,23.977082,574.900453,0.0,27.246745,82,24.544309,602.423116,0
4,2017-01-20,8195,74.942708,252,68.986674,4759.161200,0,7356.734375,30530,7054.791101,...,29.075233,99.950187,23.816345,567.218276,0.0,27.496745,82,24.803278,615.202595,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,2018-09-27,32113,282.549479,731,212.996813,45367.642454,1,46052.729167,126311,39327.219877,...,58.538935,94.226847,14.491943,210.016419,0.0,105.282552,276,71.490303,5110.863435,2
619,2018-09-28,31922,295.841146,850,226.268073,51197.241018,1,47839.028646,127145,41353.704385,...,58.976000,94.226847,14.476291,209.563007,0.0,105.413411,276,71.670375,5136.642660,2
620,2018-09-29,26340,313.122396,850,236.369667,55870.619445,1,49893.776042,131633,42208.951527,...,59.299417,94.226847,14.455909,208.973306,0.0,105.009766,276,71.264058,5078.566028,2
621,2018-09-30,27124,324.010417,850,239.438311,57330.704852,1,52994.729167,131704,43514.536764,...,59.639713,94.226847,14.381584,206.829948,0.0,103.016927,267,69.650592,4851.204925,2


In [34]:
for idx, feature in enumerate(train_fe_df.columns):
    print(f'{idx}. {feature}')

0. fecha
1. total_calls
2. mean_total_calls_historical_last_3_days
3. max_total_calls_historical_last_3_days
4. stddev_total_calls_historical_last_3_days
5. var_total_calls_historical_last_3_days
6. min_total_calls_historical_last_3_days
7. mean_total_calls_duration_last_3_days
8. max_total_calls_duration_last_3_days
9. stddev_total_calls_duration_last_3_days
10. var_total_calls_duration_last_3_days
11. min_total_calls_duration_last_3_days
12. mean_missing_calls_last_3_days
13. max_missing_calls_last_3_days
14. stddev_missing_calls_last_3_days
15. var_missing_calls_last_3_days
16. min_missing_calls_last_3_days
17. mean_available_time_last_3_days
18. max_available_time_last_3_days
19. stddev_available_time_last_3_days
20. var_available_time_last_3_days
21. min_available_time_last_3_days
22. mean_away_time_last_3_days
23. max_away_time_last_3_days
24. stddev_away_time_last_3_days
25. var_away_time_last_3_days
26. min_away_time_last_3_days
27. mean_busy_time_last_3_days
28. max_busy_time_

In [35]:
test_fe_df

,fecha,total_calls,mean_total_calls_historical_last_3_days,max_total_calls_historical_last_3_days,stddev_total_calls_historical_last_3_days,var_total_calls_historical_last_3_days,min_total_calls_historical_last_3_days,mean_total_calls_duration_last_3_days,max_total_calls_duration_last_3_days,stddev_total_calls_duration_last_3_days,...,mean_shrinkage_rate_last_15_days,max_shrinkage_rate_last_15_days,stddev_shrinkage_rate_last_15_days,var_shrinkage_rate_last_15_days,min_shrinkage_rate_last_15_days,mean_agent_headcount_last_15_days,max_agent_headcount_last_15_days,stddev_agent_headcount_last_15_days,var_agent_headcount_last_15_days,min_agent_headcount_last_15_days
0,2018-10-01,28996,324.421875,850,236.836977,56091.753672,1,54618.078125,136503,44440.273725,...,60.228841,96.949234,14.608096,213.396476,0.0,101.772135,243,68.880817,4744.566936,2
1,2018-10-02,32771,316.333333,844,228.408773,52170.567450,1,54836.539062,172368,45105.703305,...,60.715073,96.949234,14.699639,216.079385,0.0,101.797526,243,69.322266,4805.576567,2
2,2018-10-03,31422,301.838542,767,220.031507,48413.864203,1,53781.351562,172368,45326.110945,...,61.116801,96.949234,14.481836,209.723570,0.0,101.509766,243,69.190093,4787.268960,2
3,2018-10-04,30795,288.903646,710,215.106091,46270.630378,1,51725.677083,172368,44334.824149,...,61.652624,100.000000,14.368497,206.453720,0.0,101.591797,243,69.542802,4836.201340,1
4,2018-10-05,28003,297.718750,792,223.854272,50110.735313,1,49227.781250,172368,42327.703176,...,62.075104,100.000000,14.379910,206.781799,0.0,102.158203,243,70.204627,4928.689613,1
5,2018-10-06,26100,297.947917,792,223.148904,49795.433312,1,45850.682292,128908,38007.379005,...,62.339939,100.000000,14.316854,204.972311,0.0,101.938802,243,70.056535,4907.918076,1
6,2018-10-07,26690,292.041667,792,216.652670,46938.379460,1,41240.190104,126113,34170.986191,...,62.618643,100.000000,14.226426,202.391197,0.0,100.505859,243,68.883104,4744.882050,1
7,2018-10-08,27635,285.994792,792,211.775535,44848.877257,1,36711.541667,126113,30660.232420,...,62.700579,100.000000,14.415794,207.815128,0.0,99.579427,243,68.481864,4689.765674,1
8,2018-10-09,31998,270.536458,788,199.961129,39984.452975,0,34136.984375,126113,27466.320976,...,62.590788,100.000000,14.688229,215.744067,0.0,100.027344,243,68.986147,4759.088503,1
9,2018-10-10,32722,260.843750,628,192.730135,37144.905026,0,33268.208333,101056,27165.691844,...,62.387090,100.000000,14.635986,214.212092,0.0,100.315104,243,69.286026,4800.553413,1
